In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

cache_path = r"D:\TrainedModel"

tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen3-0.6B", cache_dir=cache_path)
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B", cache_dir=cache_path)

In [2]:
from datasets import load_dataset
from trl import DPOTrainer, DPOConfig
#ds = load_dataset("entfane/psychotherapy-dpo")
ds = load_dataset("hungtuchen/MentalLlama-dpo-pairs")

README.md:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

c:\Users\AriZu\.conda\envs\train\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\AriZu\.cache\huggingface\hub\datasets--hungtuchen--MentalLlama-dpo-pairs. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


dpo_pairs.json:   0%|          | 0.00/952k [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [3]:
ds = ds.rename_column("Context", "prompt")

print(ds["train"].column_names)  # 应该显示 ['prompt','chosen','rejected']

['prompt', 'chosen', 'rejected']


In [3]:
ds = ds.remove_columns("illness")
ds = ds.rename_column("input", "prompt")

print(ds["train"].column_names)  # 应该显示 ['prompt','chosen','rejected']

['prompt', 'chosen', 'rejected']


In [9]:
len(ds["train"])

352

In [4]:
from peft import get_peft_config, get_peft_model, LoraConfig

In [10]:
lora_config = LoraConfig(
    r=8,                      
    target_modules=["q_proj","v_proj"],  
    lora_alpha=32,
    lora_dropout=0.1,
    bias="none",
)
model = get_peft_model(model, lora_config)

In [11]:
training_args = DPOConfig(
    output_dir="D:\CoachBot",            
    per_device_train_batch_size=1,
    #per_device_eval_batch_size=1,
    learning_rate=1e-5,
    num_train_epochs=2,
    logging_steps=5,
    eval_steps=5,
    save_strategy="steps",
    save_steps=5,
    save_total_limit=2,               
)

trainer = DPOTrainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],   
    tokenizer=tokenizer,
)

C:\Users\AriZu\AppData\Local\Temp\ipykernel_15548\30273674.py:14: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `DPOTrainer.__init__`. Use `processing_class` instead.
  trainer = DPOTrainer(
No label_names provided for model class `PeftModel`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
trainer.train()
trainer.save_model("D:/CoachBot")

TypeError: train() got unexpected keyword arguments: load_weights_only.